In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from pyomo.opt import SolverFactory, TerminationCondition
from src.analyses.value_function import ValueFunction
from src.analyses.convergence_analysis import HausdorffAnalyzer
import numpy as np
import matplotlib.pyplot as plt
from src.models.cz_model import CaoZavalaModel, CaoZavalaAlgo
from scipy.optimize import curve_fit
def get_critical_width(alg,y_optimal,k=1,abs_epsilon=0.01):
    haus=HausdorffAnalyzer(alg)
    print(f"width: {abs_epsilon/k}")
    print(haus._gen_interval(y_optimal,abs_epsilon/k))
def linear_func(x, a, b):
        return a * x + b
def convergence_analysis(alg,fixed_pt,optimal_sol,start=-5,stop=0,steps=6,name="instance"):

    haus=HausdorffAnalyzer(alg)
    eps_list, distances=haus.analyze(y=fixed_pt, v=optimal_sol,
                    eps_min= start, eps_max = stop, steps= steps,
                    y_optimal= True)
    log_eps = np.log10(eps_list)
    log_distances = np.log10(distances)

    # Perform curve fitting
    params, _ = curve_fit(linear_func, log_eps, log_distances)
    print(f"Slope:{params[0]} intercept:{params[1]}")
    plt.loglog(eps_list, distances)
    plt.grid()
    plt.xticks(np.logspace(start, stop, steps))
    plt.xlabel("epsilon")
    plt.ylabel("Hausdorff distance")
    plt.title(name)
    plt.show()

# List of function names to be used as options
function_names = ['FLECCS', 'poutil', 'ex8_4_4',"process","pooling","crude_selection","tanksize"]
plt.style.use(['./src/utility/' + i + '.mplstyle' for i in ['font-sans', 'size-4-4', 'fontsize']])

# Define your functions here
def FLECCS():
    from NSPLIB.src.instances.FLECCS.src.pyomo_model.model import const_model
    fleccs_obj = -4.946928843629e+08  # Replace with actual value
    fleccs_sol = {'x_sorbent_total': 0.7571021770410511,'x_air_adsorb_max': 0.7567643358197031} 
    fleccs_y_bound = {'x_sorbent_total': [0.0, 1.0],"x_air_adsorb_max": [0.0, 1.0]}

    n_day = 7
    week_diff = 52
    sto_m = const_model(n_day=n_day, week_diff=week_diff)
    m = CaoZavalaModel.from_sto_m(sto_m)
    m.build()
    alg=CaoZavalaAlgo(m,solver="baron")
    convergence_analysis(alg,fleccs_sol,fleccs_obj,
                        start=-5,stop=0,steps=6,name="FLECCS")
    plt.show()
    print("FLECCS executed")

def poutil():
    from NSPLIB.src.instances.poutil.poutil_SP import const_model
    poutil_obj = 266187.5332404778  # Replace with actual value
    poutil_sol = {'alpha': 74.94473247856709, 'beta': 9.000000000000028}  # Replace with actual solution values
    poutil_y_bound = {'alpha': [0.0, 470], 'beta': [0.0, 470]}  # Replace with actual bounds
    sto_m = const_model()
    m = CaoZavalaModel.from_sto_m(sto_m)
    m.build()
    alg=CaoZavalaAlgo(m,solver="baron")
    convergence_analysis(alg,poutil_sol,poutil_obj,
                        start=-5,stop=0,steps=6,name="Poutil")
    print("Poutil executed")

def ex8_4_4():
    from NSPLIB.src.instances.ex8_4_4.ex8_4_4_SP import const_model
    ex844_obj= 0.33272388311799445
    ex844_sol = {
        'x10': -0.23129676903369037, 'x11': 1.2249985914391361, 'x12': 0.05263738880099749, 'x13': 0.5614202118628403,
        'x14': 0.6217131656002401, 'x15': 1.1, 'x16': 0.806868640168786, 'x17': 0.663161467338442, 'x6': -1.1370129865018144,
        'x7': 1.5979758364259327, 'x8': -0.5986905053355759, 'x9': 1.372365581205024
    }
    ex844_y_bound = {
        'x10': [-1.2, 0.8], 'x11': [0.1, 2.1], 'x12': [-1.1, 0.9], 'x13': [0, 1],
        'x14': [0, 1], 'x15': [1.1, 1.3], 'x16': [0, 1], 'x17': [0, 1],
        'x6': [-2, 0], 'x7': [0.5, 2.5], 'x8': [-1.5, 0.5], 'x9': [0.2, 2.2]
    }

    sto_m = const_model()
    m = CaoZavalaModel.from_sto_m(sto_m)
    m.build()

    alg=CaoZavalaAlgo(m,solver="baron")

    convergence_analysis(alg,ex844_sol,ex844_obj,start=-6,stop=-2,steps=5,name="Ex8_4_4")
    print("Ex8_4_4 executed")
def process():
    from NSPLIB.src.instances.process.process_SP import const_model
    process_obj = -1126.4218270121305
    process_sol = {"x1":1727.2601809997955,"x2":16000,"x3":104.23841082714829,"x5":2000}
    process_y_bound = {"x1":[10,2000],"x2":[0,16000],"x3":[0,120],"x5":[0,2000]}

    sto_m = const_model()
    m = CaoZavalaModel.from_sto_m(sto_m)
    m.build()
    alg=CaoZavalaAlgo(m,solver="baron")
    convergence_analysis(alg,process_sol,process_obj,start=-4,stop=0,steps=5,name="Process")
    print("Process executed")
def pooling():
    from NSPLIB.src.instances.pooling_contract_selection.pooling import const_model
    # Constants
    pooling_obj=-1338.2471283376406
    pooling_sol = {
        'A[1]': 300.0, 'A[2]': 201.92127476313524, 'A[3]': 0.0, 'A[4]': 0.0, 'A[5]': 245.18105081826008,
        'S[1]': 247.10232558139526, 'S[2]': 0.0, 'S[3]': 0.0, 'S[4]': 500.0
    }

    pooling_y_bound = {
        'A[1]': [0, 300], 'A[2]': [0, 250], 'A[3]': [0, 0], 'A[4]': [0, 0], 'A[5]': [0, 300],
        'S[1]': [0, 400], 'S[2]': [0, 0], 'S[3]': [0, 0], 'S[4]': [0, 500]
    }
    sto_m = const_model()
    m = CaoZavalaModel.from_sto_m(sto_m)
    m.build()
    binary_ys = ['lambd[1]', 'lambd[2]', 'lambd[3]', 'lambd[4]', 'lambd[5]', 'theta[1]', 'theta[2]', 'theta[3]', 'theta[4]']
    binary_y_val = {y: 0 for y in binary_ys}
    binary_y_val['lambd[1]'] = 1
    binary_y_val['lambd[2]'] = 1
    binary_y_val['lambd[5]'] = 1
    binary_y_val['theta[1]'] = 1
    binary_y_val['theta[4]'] = 1
    m.fix_binary_y(binary_y_val)

    alg=CaoZavalaAlgo(m,solver="baron")
    convergence_analysis(alg,pooling_sol,pooling_obj,start=-5,stop=0,steps=6,name="Pooling")
    print("Pooling executed")

def crude_selection():
    from NSPLIB.src.instances.crude_selection.crude_selection import const_model
    crude_obj = -18350.146929611754
    crude_sol = {f'crudeQuantity[{i}]': 0. for i in range(1, 10 + 1)}
    crude_sol['crudeQuantity[2]'] = 150.87595641747944
    crude_sol['crudeQuantity[3]'] = 201.29570746971186
    crude_sol['crudeQuantity[4]'] = 56.18456149457359
    crude_sol['crudeQuantity[8]'] = 162.2466500589715
    crude_sol['crudeQuantity[10]'] = 18.848116800048512
    crude_y_bound = {
        'crudeQuantity[1]': [0, 201.29570747217807],
        'crudeQuantity[2]': [0, 212.48012718600953],
        'crudeQuantity[3]': [0, 201.29570747217807],
        'crudeQuantity[4]': [0, 199.57869634340224],
        'crudeQuantity[5]': [0, 210.54848966613673],
        'crudeQuantity[6]': [0, 222.1383147853736],
        'crudeQuantity[7]': [0, 196.7885532591415],
        'crudeQuantity[8]': [0, 208.54531001589828],
        'crudeQuantity[9]': [0, 204.3720190779014],
        'crudeQuantity[10]': [0, 210.2623211446741]
    }

    sto_m = const_model()
    m = CaoZavalaModel.from_sto_m(sto_m)
    m.build()
    binary_ys = [f'pickCrude[{i}]' for i in range(1, 10 + 1)]
    binary_y_val = {y: 0 for y in binary_ys}
    binary_y_val['pickCrude[2]'] = 1
    binary_y_val['pickCrude[3]'] = 1
    binary_y_val['pickCrude[4]'] = 1
    binary_y_val['pickCrude[8]'] = 1
    binary_y_val['pickCrude[10]'] = 1
    m.fix_binary_y(binary_y_val)
    updated_y_bound=m.y_bound
    updated_y_bound['crudeQuantity[1]']=(0,0)
    updated_y_bound['crudeQuantity[5]']=(0,0)
    updated_y_bound['crudeQuantity[6]']=(0,0)
    updated_y_bound['crudeQuantity[7]']=(0,0)
    updated_y_bound['crudeQuantity[9]']=(0,0)
    m.update_y_bound(updated_y_bound)
    alg=CaoZavalaAlgo(m,solver="baron")
    convergence_analysis(alg,crude_sol,crude_obj,start=-5,stop=-1,steps=6,name="Crude")
    print("Crude Selection executed")
def tanksize():
    from NSPLIB.src.instances.tanksize.tanksize import const_model
    tanksize_obj = 0.9030394623070541
    tanksize_sol = {"productTankSize[1]": 659.2053849850757, "productTankSize[2]": 552.3954188143167, "productTankSize[3]": 225.73997707963284}
    tankszie_y_bound = {"productTankSize[1]": [643, 4018.36], "productTankSize[2]": [536, 3348], "productTankSize[3]": [214, 1339.45]}
    sto_m = const_model()
    m = CaoZavalaModel.from_sto_m(sto_m)
    m.build()
    alg=CaoZavalaAlgo(m,solver="baron")

    convergence_analysis(alg,tanksize_sol,tanksize_obj,start=-5,stop=0,steps=6,name="Tanksize")
    print("Tanksize executed")
    
# Dropdown widget
dropdown = widgets.Dropdown(
    options=function_names,
    description='Select Function:',
    value=function_names[5]
)

output = widgets.Output()

def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output:
            clear_output()
            # Call the function by name
            globals()[change['new']]()

dropdown.observe(on_dropdown_change)

# Call the default function when first rendered
with output:
    clear_output()
    globals()[dropdown.value]()

display(dropdown, output)